In [ ]:
import os

In [ ]:
import sklearn.cluster

In [ ]:
import gdal

In [ ]:
import numpy as np

In [ ]:
import cv2

In [ ]:
import earthpy.spatial as es
import earthpy.plot as ep
import matplotlib.pyplot as plt
import rasterio

In [ ]:
from glob import glob

In [ ]:
ndvi_path = os.getcwd() + "\\ndvi_images"

In [ ]:
print(os.path.exists(ndvi_path))

In [ ]:
ndvi_images = glob(ndvi_path + "\\*.tif")

In [ ]:
for scene in ndvi_images:
    print(scene)

In [ ]:
def saveImage(src, savePath, arr):
    with rasterio.Env():
        profile = src.profile
        profile.update(
            dtype=rasterio.float64,
            count=1)
        with rasterio.open(savePath, 'w', **profile) as dst:
            dst.write(arr.astype(rasterio.float64), 1)

In [ ]:
def imMap(date, ndvi):
    ep.plot_bands(ndvi, 
        cmap='Spectral',
        scale=False,
        title=date)
    plt.show()

In [ ]:
def hist(date, ndvi):
    ep.hist(ndvi,
            figsize=(12, 6),
            title=[date])
    plt.show()

In [ ]:
def filterImage(ndvi_image, ndvi, lower, upper):
    condition = (ndvi > lower)
    ndvi_cond = np.where(condition == 0, np.nan, ndvi)
    condition = (ndvi <= upper)
    ndvi_cond = np.where(condition == 0, np.nan, ndvi_cond)
    area = np.count_nonzero(ndvi_cond != np.nan)
    print("Pixels with NDVI > ", lower, " and <= ", upper, " = ", area)
    print("Land area with NDVI > ", lower, " and <= ", upper, " = ", area / 100, " hectares")
    #saveImage(ndvi_image, savePath, ndvi_cond)
    imMap(str(lower)+"-"+str(upper), ndvi_cond)
    return ndvi_cond

In [ ]:
for scene in ndvi_images:
    ndvi_image = gdal.Open(scene)
    ndvi = np.array(ndvi_image.GetRasterBand(1).ReadAsArray())
    imMap(scene.split('\\')[-1], ndvi)
    hist(scene.split('\\')[-1], ndvi)
    builtup = filterImage(rasterio.open(scene), ndvi, -1.01, -0.1)
    water = filterImage(rasterio.open(scene), ndvi, -0.1, 0.20)
    sparse = filterImage(rasterio.open(scene), ndvi, 0.20, 0.50)
    dense = filterImage(rasterio.open(scene), ndvi, 0.50, 1)

In [ ]:
scene = ndvi_images[-1]
ndvi_image = gdal.Open(scene)
ndvi = np.array(ndvi_image.GetRasterBand(1).ReadAsArray())
imMap(scene.split('\\')[-1], ndvi)
hist(scene.split('\\')[-1], ndvi)
ranges = [-1.01, -0.4, -0.2, -0.1, 0.3, 0.6, 1.01]
for i in range(1, len(ranges)):
    builtup = filterImage(rasterio.open(scene), ndvi, ranges[i-1], ranges[i])

In [ ]:
scene = ndvi_images[1]
ndvi_image = gdal.Open(scene)
ndvi = np.array(ndvi_image.GetRasterBand(1).ReadAsArray())
imMap(scene.split('\\')[-1], ndvi)
hist(scene.split('\\')[-1], ndvi)
ranges = [-1.01, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.01]

builtup = filterImage(rasterio.open(scene), ndvi, -1.01, -0.1)
water = filterImage(rasterio.open(scene), ndvi, -0.1, 0.20)
sparse = filterImage(rasterio.open(scene), ndvi, 0.20, 0.50)
dense = filterImage(rasterio.open(scene), ndvi, 0.50, 1)

In [ ]:
scene = ndvi_images[1]
ndvi_image = gdal.Open(scene)
ndvi = np.array(ndvi_image.GetRasterBand(1).ReadAsArray())

In [ ]:
vectorized = ndvi.reshape((-1,3))

In [ ]:
vectorized = np.float32(vectorized)

In [ ]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

In [ ]:
K = 3
attempts=10
ret,label,center=cv2.kmeans(vectorized,K,None,criteria,attempts,cv2.KMEANS_PP_CENTERS)